Análise Inicial dos dados

In [9]:
# importa biblioteca pandas
import pandas as pd
# importa biblioteca numpy
import numpy as np
# importa biblioteca statsmodels
import statsmodels.api as sm

# local do arquivo
Caminho='Credito.xlsx'

KS >= 30
AUC >= 1.25 x acaso
ROC >= 60

In [10]:
#Traz o arquivo para o Python
df=pd.read_excel(Caminho)
df.head()

,id,idade,qtd_com,tempo_em,sal_cli,tempo_res,qtd_parc,vlr_cpr,vlr_parc,tipo_cre,tipo
0,1,57,1,238,4096.9,322,8,2000.0,394.22,1,1
1,2,40,0,24,581.3,300,9,300.0,64.34,1,0
2,3,28,1,72,901.5,216,6,700.0,176.50,1,0
3,4,41,1,25,1279.0,76,8,1200.0,245.29,1,1
4,5,44,0,120,3000.0,132,6,600.0,170.25,1,0


In [11]:
#Tabelas de Frequencia para variáveis binárias/nominais

# Lista de variáveis para as quais você deseja calcular tabelas de frequência
variaveis = [  'qtd_com',  'tipo_cre','tipo']

# Calcular tabelas de frequência para cada variável
for variavel in variaveis:
    freq_table = df[variavel].value_counts()
    percentual = (df[variavel].value_counts(normalize=True) * 100).round(2)

    # Concatenar as tabelas de frequência e percentual
    freq_table = pd.concat([freq_table, percentual], axis=1)
    freq_table.columns = ['Frequência', 'Percentual (%)']

    print("Tabela de Frequência para '{}':\n".format(variavel), freq_table)
    print("\n")


Tabela de Frequência para 'qtd_com':
          Frequência  Percentual (%)
qtd_com                            
0              9852           54.73
1              8148           45.27


Tabela de Frequência para 'tipo_cre':
           Frequência  Percentual (%)
tipo_cre                            
1              15307           85.04
0               2693           14.96


Tabela de Frequência para 'tipo':
       Frequência  Percentual (%)
tipo                            
1           9000            50.0
0           9000            50.0




In [12]:
#Análises descritivas variáveis quantitativas

def analise_descritiva(df, variaveis):
    # Estatísticas descritivas para variáveis numéricas
    desc_stats_numericas = df[variaveis].describe()

    return desc_stats_numericas

def main():
    # Variáveis para análise descritiva
    variaveis_para_analise = ['idade',  'tempo_em', 'sal_cli', 'tempo_res', 'qtd_parc', 'vlr_cpr', 'vlr_parc']

    # Realizar análise descritiva
    desc_stats_numericas = analise_descritiva(df, variaveis_para_analise)

    # Imprimir resultados
    print("Estatísticas Descritivas para Variáveis Numéricas:")
    print(desc_stats_numericas)

if __name__ == "__main__":
    main()

Estatísticas Descritivas para Variáveis Numéricas:
              idade      tempo_em        sal_cli     tempo_res      qtd_parc  \
count  18000.000000  18000.000000   17998.000000  18000.000000  18000.000000   
mean      42.896278    102.019278    1572.563755    160.166000      7.910333   
std       13.470339     93.292373    1955.513545    141.291454      3.099112   
min       18.000000      0.000000       0.010000      0.000000      1.000000   
25%       32.000000     31.000000     722.082500     46.000000      6.000000   
50%       41.000000     72.000000    1109.230000    120.000000      8.000000   
75%       52.000000    144.000000    1791.922500    245.000000     12.000000   
max       88.000000   1176.000000  158600.000000    960.000000     12.000000   

            vlr_cpr      vlr_parc  
count  18000.000000  18000.000000  
mean     902.100776    205.121338  
std      811.889030    174.457318  
min      150.000000     40.860000  
25%      400.000000    112.465000  
50%      600

In [13]:
#Como o menor salário é 0.01, vamos fazer um ajuste e colocar um valor mais factível como mínimo por exemplo R$ 250

df['sal_cli'] = df['sal_cli'].apply(lambda x: 250 if pd.isnull(x) or x < 250 else x)

# Criar nova variável dividindo 'vlr_parc' por 'sal_cli' - Comprometimento
df['comprom'] = df['vlr_parc'] / df['sal_cli']


In [14]:
#amostra estratificada
from sklearn.model_selection import train_test_split

def criar_amostra_estratificada(df, variavel_resposta, tamanho_amostra, variaveis_preditoras):
    X = df[variaveis_preditoras]
    y = df[variavel_resposta]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tamanho_amostra, stratify=y, random_state=42)
    return X_train, X_test, y_train, y_test

# Definição das variáveis preditoras para análise
variaveis_para_analise = ['idade', 'qtd_com', 'tempo_em', 'sal_cli', 'tempo_res', 'qtd_parc', 'vlr_cpr', 'vlr_parc', 'tipo_cre','comprom']

# Chamada da função para criar amostra estratificada
X_train, X_test, y_train, y_test = criar_amostra_estratificada(df, 'tipo', 0.4, variaveis_para_analise)

# Imprimir informações sobre a amostra
print("\nInformações sobre a amostra estratificada:")
print("Tamanho do conjunto de treino:", len(X_train))
print("Tamanho do conjunto de teste:", len(X_test))


# Calcular frequência da variável 'tipo' nas amostras de treino e teste
frequencia_tipo_treino = y_train.value_counts()
frequencia_tipo_teste = y_test.value_counts()

# Imprimir frequência da variável 'tipo'
print("Frequência da variável 'tipo' no conjunto de treino:")
print(frequencia_tipo_treino)

print("\nFrequência da variável 'tipo' no conjunto de teste:")
print(frequencia_tipo_teste)


Informações sobre a amostra estratificada:
Tamanho do conjunto de treino: 10800
Tamanho do conjunto de teste: 7200
Frequência da variável 'tipo' no conjunto de treino:
tipo
1    5400
0    5400
Name: count, dtype: int64

Frequência da variável 'tipo' no conjunto de teste:
tipo
1    3600
0    3600
Name: count, dtype: int64


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Criar pipeline com modelo de regressão logística
pipeline = Pipeline([
    ('logistic_regression', LogisticRegression())
])

# Treinar o modelo no pipeline usando os dados de treino
pipeline.fit(X_train, y_train)

# Coeficientes do modelo de regressão logística
coeficientes = pipeline.named_steps['logistic_regression'].coef_[0]

# Intercepto do modelo de regressão logística
intercepto = pipeline.named_steps['logistic_regression'].intercept_

# Nomes das variáveis preditoras
nomes_variaveis = X_train.columns.tolist()

# Imprimir coeficientes associados a cada variável
print("Coeficientes do modelo:")
for nome_variavel, coeficiente in zip(nomes_variaveis, coeficientes):
    print(nome_variavel, ":", coeficiente)

# Imprimir intercepto do modelo
print("\nIntercepto do modelo:", intercepto)

Coeficientes do modelo:
idade : 0.016312758022799623
qtd_com : 0.8409731591401389
tempo_em : 0.0015641375592017424
sal_cli : 2.0057185859914225e-05
tempo_res : 0.0001062054303791891
qtd_parc : -0.23958372323792929
vlr_cpr : 0.00020408853059889786
vlr_parc : -0.0014172278393340533
tipo_cre : 0.7491838002858225
comprom : 0.005085309409442522

Intercepto do modelo: [0.1015549]


C:\Users\gugat\PycharmProjects\PUC\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

# Prever as probabilidades das classes positivas para os dados de teste
probabilidades_positivas = pipeline.predict_proba(X_test)[:, 1]

# Calcular KS
fpr, tpr, thresholds = roc_curve(y_test, probabilidades_positivas)
ks = max(tpr - fpr)

# Calcular AUC (área sob a curva ROC)
auc = roc_auc_score(y_test, probabilidades_positivas)

# Calcular matriz de confusão
previsoes = pipeline.predict(X_test)
matriz_confusao = confusion_matrix(y_test, previsoes)

# Calcular acurácia total
acuracia_total = pipeline.score(X_test, y_test)

# Imprimir métricas de avaliação
print("KS:", ks)
print("AUC:", auc)
print("Matriz de Confusão:")
print(matriz_confusao)
print("Acurácia Total:", acuracia_total)

KS: 0.35944444444444446
AUC: 0.7325966820987655
Matriz de Confusão:
[[2451 1149]
 [1202 2398]]
Acurácia Total: 0.6734722222222222


In [17]:
from sklearn.neural_network import MLPClassifier

# Criar modelo de redes neurais
modelo_rede_neural = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)

# Treinar o modelo nos dados de treino
modelo_rede_neural.fit(X_train, y_train)

# Avaliar o modelo nos dados de teste
acuracia_teste_rede_neural = modelo_rede_neural.score(X_test, y_test)

# Imprimir acurácia do modelo nos dados de teste
print("Acurácia do modelo de redes neurais nos dados de teste:", acuracia_teste_rede_neural)

Acurácia do modelo de redes neurais nos dados de teste: 0.5322222222222223


In [18]:
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

# Prever as probabilidades das classes positivas para os dados de teste
probabilidades_positivas_rede_neural = modelo_rede_neural.predict_proba(X_test)[:, 1]

# Calcular KS
fpr_rede_neural, tpr_rede_neural, thresholds_rede_neural = roc_curve(y_test, probabilidades_positivas_rede_neural)
ks_rede_neural = max(tpr_rede_neural - fpr_rede_neural)

# Calcular AUC (área sob a curva ROC)
auc_rede_neural = roc_auc_score(y_test, probabilidades_positivas_rede_neural)

# Calcular matriz de confusão
previsoes_rede_neural = modelo_rede_neural.predict(X_test)
matriz_confusao_rede_neural = confusion_matrix(y_test, previsoes_rede_neural)

# Calcular acurácia total
acuracia_total_rede_neural = modelo_rede_neural.score(X_test, y_test)

# Imprimir métricas de avaliação para o modelo de rede neural
print("KS (Rede Neural):", ks_rede_neural)
print("AUC (Rede Neural):", auc_rede_neural)
print("Matriz de Confusão (Rede Neural):")
print(matriz_confusao_rede_neural)
print("Acurácia Total (Rede Neural):", acuracia_total_rede_neural)

KS (Rede Neural): 0.2386111111111111
AUC (Rede Neural): 0.6598560570987654
Matriz de Confusão (Rede Neural):
[[ 297 3303]
 [  65 3535]]
Acurácia Total (Rede Neural): 0.5322222222222223


In [19]:
from sklearn.tree import DecisionTreeClassifier

# Criar modelo de árvore de decisão
modelo_arvore = DecisionTreeClassifier(random_state=42)

# Treinar o modelo nos dados de treino
modelo_arvore.fit(X_train, y_train)

# Avaliar o modelo nos dados de teste
acuracia_teste_arvore = modelo_arvore.score(X_test, y_test)

# Imprimir acurácia do modelo nos dados de teste
print("Acurácia do modelo de árvore de decisão nos dados de teste:", acuracia_teste_arvore)

Acurácia do modelo de árvore de decisão nos dados de teste: 0.6027777777777777


In [20]:
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

# Prever as probabilidades das classes positivas para os dados de teste
probabilidades_positivas_arvore = modelo_arvore.predict_proba(X_test)[:, 1]

# Calcular KS
fpr_arvore, tpr_arvore, thresholds_arvore = roc_curve(y_test, probabilidades_positivas_arvore)
ks_arvore = max(tpr_arvore - fpr_arvore)

# Calcular AUC (área sob a curva ROC)
auc_arvore = roc_auc_score(y_test, probabilidades_positivas_arvore)

# Calcular matriz de confusão
previsoes_arvore = modelo_arvore.predict(X_test)
matriz_confusao_arvore = confusion_matrix(y_test, previsoes_arvore)

# Calcular acurácia total
acuracia_total_arvore = modelo_arvore.score(X_test, y_test)

# Imprimir métricas de avaliação para o modelo de árvore de decisão
print("KS (Árvore de Decisão):", ks_arvore)
print("AUC (Árvore de Decisão):", auc_arvore)
print("Matriz de Confusão (Árvore de Decisão):")
print(matriz_confusao_arvore)
print("Acurácia Total (Árvore de Decisão):", acuracia_total_arvore)


KS (Árvore de Decisão): 0.20555555555555555
AUC (Árvore de Decisão): 0.6027777777777779
Matriz de Confusão (Árvore de Decisão):
[[2211 1389]
 [1471 2129]]
Acurácia Total (Árvore de Decisão): 0.6027777777777777


In [21]:
from sklearn.ensemble import RandomForestClassifier

# Criar modelo Random Forest
modelo_random_forest = RandomForestClassifier(random_state=42)

# Treinar o modelo nos dados de treino
modelo_random_forest.fit(X_train, y_train)

# Avaliar o modelo nos dados de teste
acuracia_teste_random_forest = modelo_random_forest.score(X_test, y_test)

# Prever as probabilidades das classes positivas para os dados de teste
probabilidades_positivas_random_forest = modelo_random_forest.predict_proba(X_test)[:, 1]

# Calcular KS
fpr_random_forest, tpr_random_forest, thresholds_random_forest = roc_curve(y_test, probabilidades_positivas_random_forest)
ks_random_forest = max(tpr_random_forest - fpr_random_forest)

# Calcular AUC (área sob a curva ROC)
auc_random_forest = roc_auc_score(y_test, probabilidades_positivas_random_forest)

# Calcular matriz de confusão
previsoes_random_forest = modelo_random_forest.predict(X_test)
matriz_confusao_random_forest = confusion_matrix(y_test, previsoes_random_forest)

# Calcular acurácia total
acuracia_total_random_forest = modelo_random_forest.score(X_test, y_test)

# Imprimir acurácia do modelo nos dados de teste
print("Acurácia do modelo Random Forest nos dados de teste:", acuracia_teste_random_forest)
print("KS (Random Forest):", ks_random_forest)
print("AUC (Random Forest):", auc_random_forest)
print("Matriz de Confusão (Random Forest):")
print(matriz_confusao_random_forest)
print("Acurácia Total (Random Forest):", acuracia_total_random_forest)


Acurácia do modelo Random Forest nos dados de teste: 0.6556944444444445
KS (Random Forest): 0.31166666666666665
AUC (Random Forest): 0.7207170910493828
Matriz de Confusão (Random Forest):
[[2477 1123]
 [1356 2244]]
Acurácia Total (Random Forest): 0.6556944444444445


Escorando uma base com o melhor modelo

In [24]:
# Carregar a nova base de clientes
caminho_nova_base = 'Credito_escorar.xlsx'
nova_base = pd.read_excel(caminho_nova_base)

# Salvar os IDs dos clientes separadamente
ids_clientes = nova_base['id']

# Remover as colunas irrelevantes ('id', 'tipo') e reordenar as colunas para corresponder à ordem do modelo
colunas_modelo = ['idade', 'qtd_com', 'tempo_em', 'sal_cli', 'tempo_res', 'qtd_parc', 'vlr_cpr', 'vlr_parc', 'tipo_cre']
nova_base = nova_base[colunas_modelo]

# Aplicar as transformações necessárias nas outras colunas
nova_base['sal_cli'] = nova_base['sal_cli'].apply(lambda x: 250 if pd.isnull(x) or x < 250 else x)
nova_base['comprom'] = nova_base['vlr_parc'] / nova_base['sal_cli']

# Fazer previsões e obter probabilidades na nova base de clientes com as transformações aplicadas
previsoes_probabilidades_nova_base = pipeline.predict_proba(nova_base)

# Extrair as previsões e as probabilidades associadas
probabilidades_nova_base = previsoes_probabilidades_nova_base[:, 1]  # Probabilidades para a classe positiva (ou 1)

# Adicionar as previsões e as probabilidades como novas colunas na nova base de clientes
nova_base['probabilidade'] = probabilidades_nova_base

# Juntar os IDs dos clientes de volta à nova base de clientes
nova_base['id'] = ids_clientes

# Salvar a nova base de clientes com as previsões
caminho_saida = 'Credito_escorada.xlsx'
nova_base.to_excel(caminho_saida, index=False)

In [26]:
nova_base = pd.read_excel(caminho_nova_base)

# Salvar os IDs dos clientes separadamente
ids_clientes = nova_base['id']

# Remover as colunas irrelevantes ('id', 'tipo') e reordenar as colunas para corresponder à ordem do modelo
colunas_modelo = ['idade',  'qtd_com', 'tempo_em', 'sal_cli', 'tempo_res', 'qtd_parc', 'vlr_cpr', 'vlr_parc', 'tipo_cre']
nova_base = nova_base[colunas_modelo]

# Aplicar as transformações necessárias nas outras colunas
nova_base['sal_cli'] = nova_base['sal_cli'].apply(lambda x: 250 if pd.isnull(x) or x < 250 else x)
nova_base['comprom'] = nova_base['vlr_parc'] / nova_base['sal_cli']

# Fazer previsões e obter probabilidades na nova base de clientes com as transformações aplicadas
previsoes_probabilidades_nova_base = modelo_random_forest.predict_proba(nova_base)

# Extrair as previsões e as probabilidades associadas
probabilidades_nova_base = previsoes_probabilidades_nova_base[:, 1]  # Probabilidades para a classe positiva (ou 1)

# Adicionar as previsões e as probabilidades como novas colunas na nova base de clientes
nova_base['probabilidade'] = probabilidades_nova_base

# Juntar os IDs dos clientes de volta à nova base de clientes
nova_base['id'] = ids_clientes


# Salvar a nova base de clientes com as previsões
caminho_saida = 'Credito_escorar.xlsx'
nova_base.to_excel(caminho_saida, index=False)